In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('trading_data.csv')
df=pd.DataFrame(df)

In [5]:
#drop far price and near price
df["near_price"]=df["near_price"].fillna(df["reference_price"])
df["far_price"]=df["far_price"].fillna(df["reference_price"])

In [11]:
df["stock_id"]=df["stock_id"].astype("category")

In [13]:
#add stockWap(t=60) column
df['future_wap'] = df.groupby(['stock_id', 'date_id'])['wap'].shift(-6)

In [10]:
sorted=df.sort_values(by=["stock_id","date_id","seconds_in_bucket"])
sorted.head(10)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,future_wap
0,0,0,0,3180602.69,1,0.999812,13380276.64,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0,1.000517
191,0,0,10,1299772.70,1,1.000026,15261106.63,0.999812,13996.50,1.000026,23519.16,0.999892,0.389814,1,0_10_0,1.000421
382,0,0,20,1299772.70,1,0.999919,15261106.63,0.999812,4665.50,0.999919,12131.60,0.999842,4.220009,2,0_20_0,1.000148
573,0,0,30,1299772.70,1,1.000133,15261106.63,1.000026,55998.00,1.000133,46203.30,1.000085,5.450249,3,0_30_0,1.000426
764,0,0,40,1218204.43,1,1.000455,15342674.90,1.000241,14655.95,1.000455,26610.45,1.000317,3.169775,4,0_40_0,1.000261
955,0,0,50,1218204.43,1,1.000455,15342674.90,1.000348,42012.00,1.000455,9897.22,1.000434,0.599623,5,0_50_0,1.000042
1146,0,0,60,1218204.43,1,1.000562,15342674.90,1.000455,14005.50,1.000562,10085.04,1.000517,-0.200272,6,0_60_0,0.999895
1337,0,0,70,1264494.89,1,1.000455,15352380.96,1.000348,37904.16,1.000455,17366.82,1.000421,2.410412,7,0_70_0,0.999962
1528,0,0,80,1189832.86,1,1.000241,15427043.00,1.000133,9427.34,1.000241,61984.40,1.000148,-0.389814,8,0_80_0,0.999826
1719,0,0,90,1189272.89,1,1.000562,15427602.97,1.000348,23340.00,1.000562,40433.54,1.000426,-4.339814,9,0_90_0,0.999787


In [14]:
train=df[df["date_id"]<478]
test = df[df["date_id"] >= 478]

In [20]:
test1=test.dropna()
df=train.dropna()

In [26]:
x_train = df.drop(columns=['future_wap', "wap", 'row_id',"target"])
y_train = df['target']

In [27]:
#
lr_model = LinearRegression()#1st model
lr_model.fit(x_train, y_train)
y_train_pred = lr_model.predict(x_train)
residuals = y_train - y_train_pred#residual from lr
residual_model=RandomForestRegressor()
residual_model.fit(x_train, residuals)

RandomForestRegressor()

In [31]:
# Make predictions with both models
x_test = test1.drop(columns=['future_wap', "wap", 'row_id','target'])
y_test = test1['target']
primary_predictions = lr_model.predict(x_test)
residual_predictions = residual_model.predict(x_test)
final_predictions = primary_predictions + residual_predictions

In [32]:
mae = mean_absolute_error(y_test, final_predictions)
mae

5.5466343437287176

In [34]:
#
lr_model = LinearRegression()#1st model
lr_model.fit(x_train, y_train)
y_train_pred = lr_model.predict(x_train)
residuals = y_train - y_train_pred#residual from lr
x_train["stock_id"] = x_train["stock_id"].cat.codes
residual_model=xgb.XGBRegressor()
residual_model.fit(x_train, residuals)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [35]:
# Make predictions with both models
x_test = test1.drop(columns=['future_wap', "wap", 'row_id','target'])
y_test = test1['target']
primary_predictions = lr_model.predict(x_test)
residual_predictions = residual_model.predict(x_test)
final_predictions = primary_predictions + residual_predictions

In [36]:
mae = mean_absolute_error(y_test, final_predictions)
mae

5.41437383258953

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
#feature fix 

In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    x_test=test.drop(columns=['future_wap', "wap", 'row_id',"target","far_price","near_price","currently_scored"])
    x_test["stock_id"=x_test["stock_id"].astype("category") 
    sample_prediction['target'] = 0
    env.predict(sample_prediction)
    counter += 1